In [2]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

## Building the Word Vocabulary

You will first download the lyrics dataset. These will be from a compilation of traditional Irish songs and you can see them [here](https://github.com/https-deeplearning-ai/tensorflow-1-public/blob/main/C3/W4/misc/Laurences_generated_poetry.txt).

In [3]:
!pip install gdown==5.1.0

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
# Download the dataset
!gdown --id 15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT

C:\Users\malat\.conda\envs\scientificProject\lib\site-packages\gdown\__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=15UqmiIm0xwh9mt0IYq2z3jHaauxQSTQT
To: D:\KrsnaPycharmProjects\scientificProject\irish-lyrics-eof.txt

  0%|          | 0.00/69.0k [00:00<?, ?B/s]
100%|##########| 69.0k/69.0k [00:00<00:00, 1.58MB/s]


Next, you will lowercase and split the plain text into a list of sentences:

In [5]:
# Load the dataset
with open('./irish-lyrics-eof.txt') as f:
  ours_corpus=f.read().lower().split('\n')

data = open('./irish-lyrics-eof.txt').read()

# Lowercase and split the text
corpus = data.lower().split("\n")

print(type(ours_corpus),type(corpus))
# Preview the result
print(corpus)
print(ours_corpus)


<class 'list'> <class 'list'>
['come all ye maidens young and fair', 'and you that are blooming in your prime', 'always beware and keep your garden fair', 'let no man steal away your thyme', 'for thyme it is a precious thing', 'and thyme brings all things to my mind', 'nlyme with all its flavours, along with all its joys', 'thyme, brings all things to my mind', 'once i and a bunch of thyme', 'i thought it never would decay', 'then came a lusty sailor', 'who chanced to pass my way', 'and stole my bunch of thyme away', 'the sailor gave to me a rose', 'a rose that never would decay', 'he gave it to me to keep me reminded', 'of when he stole my thyme away', 'sleep, my child, and peace attend thee', 'all through the night', 'guardian angels god will send thee', 'soft the drowsy hours are creeping', 'hill and dale in slumber sleeping', 'i my loving vigil keeping', 'while the moon her watch is keeping', 'while the weary world is sleeping', 'oer thy spirit gently stealing', 'visions of delight

In [6]:
import functools
if functools.reduce(lambda x, y : x and y, map(lambda p, q: p == q,ours_corpus,corpus), True):
  print ("The lists l1 and l2 are the same")
else:
  print ("The lists l1 and l2 are not the same")


The lists l1 and l2 are the same


In [7]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(ours_corpus)
total_words=len(tokenizer.word_index)+1
seq_zero_ind_corpus=tokenizer.texts_to_sequences([ours_corpus[0]])
print('total_words',total_words,'seq_zero_ind_corpus',seq_zero_ind_corpus)
print(type(tokenizer.word_index))

total_words 2690 seq_zero_ind_corpus [[51, 12, 96, 1217, 48, 2, 69]]
<class 'dict'>


In [8]:
li_seq=[]
li=[]
for cnt,line in enumerate(ours_corpus):
  seq=tokenizer.texts_to_sequences([line])
  seq_1dim_less=seq[0]
  li_seq.append(seq_1dim_less)
  maxlen=max([len(i) for i in li_seq])
  for i in range(1,len(seq_1dim_less)):
    li.append(seq_1dim_less[:i+1])

  # if(cnt<2):
    # print(len(seq_1dim_less))

    # print(cnt,line)
    # print(li_seq)
    # print(li)
    # print('maxlen',maxlen)

pd_li=pad_sequences(li,maxlen=maxlen,padding='pre')
print(pd_li[:5])
print(pd_li.shape)
xs=pd_li[:,:-1]
lbls=pd_li[:,-1]
xs_0=pd_li[0][:-1]
# lbls=pd_li[:,-1]
print(xs.shape)
print(xs[:5])
print(lbls.shape)
print(lbls[:5])
print('maxlen',maxlen)

ys=tf.keras.utils.to_categorical(lbls,num_classes=total_words)
print(ys.shape)
print(ys[:5])


[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    51   12]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   51
    12   96]
 [   0    0    0    0    0    0    0    0    0    0    0    0   51   12
    96 1217]
 [   0    0    0    0    0    0    0    0    0    0    0   51   12   96
  1217   48]
 [   0    0    0    0    0    0    0    0    0    0   51   12   96 1217
    48    2]]
(12038, 16)
(12038, 15)
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    51]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0   51
    12]
 [   0    0    0    0    0    0    0    0    0    0    0    0   51   12
    96]
 [   0    0    0    0    0    0    0    0    0    0    0   51   12   96
  1217]
 [   0    0    0    0    0    0    0    0    0    0   51   12   96 1217
    48]]
(12038,)
[  12   96 1217   48    2]
maxlen 16
(12038, 2690)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ...

In [9]:
embedding_dim=128

In [10]:
def create_model_seq():
  model=Sequential([
      Embedding(total_words,embedding_dim,input_length=maxlen-1),
      Bidirectional(LSTM(128)),
      Dense(total_words,activation='softmax')
      # Dense(64,activation='relu')
  ])
  return model

In [11]:
model=create_model_seq()

In [12]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [13]:
epochs=500

In [15]:
history=model.fit(xs,ys,epochs=epochs)

Epoch 1/500
377/377 [==============================] - 11s 5ms/step - loss: 6.7526 - accuracy: 0.0643
Epoch 2/500
377/377 [==============================] - 2s 5ms/step - loss: 6.2358 - accuracy: 0.0753
Epoch 3/500
377/377 [==============================] - 2s 5ms/step - loss: 5.9655 - accuracy: 0.0894
Epoch 4/500
377/377 [==============================] - 2s 5ms/step - loss: 5.6457 - accuracy: 0.1069
Epoch 5/500
377/377 [==============================] - 2s 5ms/step - loss: 5.3160 - accuracy: 0.1177
Epoch 6/500
377/377 [==============================] - 2s 5ms/step - loss: 4.9892 - accuracy: 0.1393
Epoch 7/500
377/377 [==============================] - 2s 5ms/step - loss: 4.6740 - accuracy: 0.1570
Epoch 8/500
377/377 [==============================] - 2s 5ms/step - loss: 4.3661 - accuracy: 0.1812
Epoch 9/500
377/377 [==============================] - 2s 5ms/step - loss: 4.0639 - accuracy: 0.2146
Epoch 10/500
377/377 [==============================] - 2s 5ms/step - loss: 3.7719 - accur

In [20]:
inp_sample='running through the valley'

In [19]:
def next_word(inp_sample):
  inp_seq=tokenizer.texts_to_sequences([inp_sample])
  pd_inp_seq=pad_sequences(inp_seq,maxlen=maxlen-1,padding='pre')
  print(pd_inp_seq)
  y_proba=model.predict(pd_inp_seq)
  print('y_proba.shape',y_proba.shape)
  y_pred=np.argmax(y_proba,axis=-1)
  print('y_pred.shape',y_pred.shape)
  print('y_pred',y_pred)
  print('y_pred[0]',y_pred[0])
  # tf.random.categorical()
  nxt_w=tokenizer.index_word[y_pred[0]]
  return nxt_w

# next_word(inp_sample)

[[  0   0   0   0   0   0   0   0   0   0   0   0  88   1 432]]
1/1 [==============================] - 0s 15ms/step
y_proba.shape (1, 2690)
y_pred.shape (1,)


'of'

In [22]:
def extend_word(inp_sample,n=2):
  for _ in range(n):
    print('inp_sample',inp_sample)
    inp_sample += ' '+next_word(inp_sample)
    print('next_word',next_word)
    print('inp_sample',inp_sample)
  
extend_word(inp_sample,50)

inp_sample running through the valley
[[  0   0   0   0   0   0   0   0   0   0   0   0  88   1 432]]
1/1 [==============================] - 0s 14ms/step
y_proba.shape (1, 2690)
y_pred.shape (1,)
next_word <function next_word at 0x000002B317647E20>
inp_sample running through the valley of
inp_sample running through the valley of
[[  0   0   0   0   0   0   0   0   0   0   0  88   1 432   6]]
1/1 [==============================] - 0s 15ms/step
y_proba.shape (1, 2690)
y_pred.shape (1,)
next_word <function next_word at 0x000002B317647E20>
inp_sample running through the valley of the
inp_sample running through the valley of the
[[  0   0   0   0   0   0   0   0   0   0  88   1 432   6   1]]
1/1 [==============================] - 0s 15ms/step
y_proba.shape (1, 2690)
y_pred.shape (1,)
next_word <function next_word at 0x000002B317647E20>
inp_sample running through the valley of the maiden
inp_sample running through the valley of the maiden
[[  0   0   0   0   0   0   0   0   0  88   1 432   6